<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

Matplotlib is building the font cache; this may take a moment.


In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 09:25:41
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: -306.00 (-0.0%)
Current PnL: -23.30 L (-15.67%)
CY Booked + Current PnL: -11.86 L (-7.98%)
-------------------
Total profit:  80.58 K
Total loss:  -24.11 L
-------------------
Total Booked + Current PnL: 15.12 L (12.42%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.39%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 89.17 L (65.43%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,91.89,34.0,M-SC,2.43,82694.0,-18083.0,18176.0,-0.58,-17.94,21.98,0.09,245.0,-0.99,0.61,8.64,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.41,44.0,H-MC,3.59,187220.0,22550.0,20781.0,-0.95,13.69,11.10,26.31,89.0,1.09,1.37,22.77,X40N,NTT,AC
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107786.0,7282.0,21827.0,1.39,7.25,20.25,28.96,79.0,0.33,0.79,43.67,MH,ATH,METALS
57,RELIANCE,1533.00,-13.66,47.0,H-LC,3.94,215842.0,5476.0,23311.0,0.44,2.60,10.80,13.68,37.0,0.23,1.58,19.56,XY25,NTT,REFINERIES
42,ITC,452.00,-38.77,44.0,H-LC,1.93,197556.0,-2582.0,23470.0,-0.27,-1.29,11.88,10.44,4.0,-0.11,1.45,4.20,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-177.27,69.0,M-SC,6.56,88446.0,-13764.0,107152.0,6.96,-13.47,121.15,91.37,208.0,-0.13,0.65,43.58,XY25,NTT,FINANCE
64,SHALBY,327.00,1185.49,65.0,M-SC,13.99,174740.0,-6403.0,51863.0,2.03,-3.53,29.68,25.10,235.0,-0.12,1.28,40.04,XY24,NTT,HEALTHCARE
32,HINDZINC,730.22,22.12,55.0,M-LC,2.90,198817.0,-6259.0,118833.0,1.77,-3.05,59.77,54.89,52.0,-0.05,1.46,20.31,X5K,ATH,METALS
75,TITAGARH,1548.00,-4.73,50.0,H-SC,3.26,203615.0,-38562.0,149331.0,1.57,-15.92,73.34,45.74,158.0,-0.26,1.49,32.43,XY24,NTT,ENGINEERING
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107786.0,7282.0,21827.0,1.39,7.25,20.25,28.96,79.0,0.33,0.79,43.67,MH,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-49.70,38.0,L-SC,40.02,75610.0,-37939.0,77992.0,-2.60,-33.41,103.15,35.27,268.0,-0.49,0.55,85.06,XR,NTT,HOTELS
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,7.00,206166.0,-45602.0,98815.0,-2.55,-18.11,47.93,21.13,27.0,-0.46,1.51,10.90,X40,ATH,PAINTS
31,HINDUNILVR,2922.00,-12.79,31.0,H-LC,8.91,244900.0,-12163.0,47290.0,-2.50,-4.73,19.31,13.67,24.0,-0.26,1.80,14.71,XY25,NTT,FMCG
38,INDIGOPNTS,1408.00,102.01,30.0,M-SC,1.52,132444.0,-42115.0,42144.0,-2.06,-24.13,31.82,0.02,221.0,-1.00,0.97,15.20,OX40N,NTT,PAINTS
13,BERGEPAINT,680.00,-22.55,35.0,H-MC,4.24,208109.0,-19230.0,67282.0,-1.46,-8.46,32.33,21.14,106.0,-0.29,1.53,17.30,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.0,-45.59,29.0,M-SC,1.39,91988.0,490.0,78263.0,-0.49,0.53,85.08,86.07,223.0,0.01,0.68,33.58,XR,NTT,DURABLES
11,BANKINDIA,190.0,-30.43,52.0,H-MC,8.40,180869.0,1061.0,111343.0,1.09,0.59,61.56,62.52,88.0,0.01,1.33,34.88,XR,NTT,BANKS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,87.86,44.0,H-SC,10.19,118800.0,-23679.0,43196.0,-0.41,-16.62,36.36,13.70,163.0,-0.55,0.87,40.22,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,91.89,34.0,M-SC,2.43,82694.0,-18083.0,18176.0,-0.58,-17.94,21.98,0.09,245.0,-0.99,0.61,8.64,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,48.0,H-SC,1.52,219771.0,-49896.0,86238.0,-0.97,-18.50,39.24,13.47,138.0,-0.58,1.61,11.16,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,36.0,H-SC,1.70,140852.0,-35051.0,77074.0,-0.90,-19.93,54.72,23.89,149.0,-0.45,1.03,21.64,OX40N,NTT,CERAMICS
66,SIS,528.0,2070.98,38.0,H-SC,4.01,87605.0,-23427.0,47035.0,-0.48,-21.10,53.69,21.26,156.0,-0.50,0.64,18.36,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,54.0,M-MC,10.71,227769.0,2807.0,164836.0,0.95,1.25,72.37,74.53,192.0,0.02,1.67,33.90,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,8.41,44.0,H-MC,3.59,187220.0,22550.0,20781.0,-0.95,13.69,11.10,26.31,89.0,1.09,1.37,22.77,X40N,NTT,AC
84,VOLTAS,1530.00,-4.30,38.0,H-MC,2.81,200880.0,9138.0,28625.0,-0.07,4.77,14.25,19.69,99.0,0.32,1.47,11.89,XY25,NTT,AC
17,CAMS,5211.76,-6.49,42.0,H-SC,1.81,106280.0,4276.0,39653.0,-0.11,4.19,37.31,43.06,122.0,0.11,0.78,22.03,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,51.0,M-LC,6.98,153415.0,12575.0,31910.0,0.63,8.93,20.80,31.59,66.0,0.39,1.13,37.78,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107786.0,7282.0,21827.0,1.39,7.25,20.25,28.96,79.0,0.33,0.79,43.67,MH,ATH,METALS
37,INDIAMART,4850.92,-55.01,26.0,H-SC,9.99,125398.0,2062.0,131706.0,-0.03,1.67,105.03,108.45,119.0,0.02,0.92,24.40,AR,ATH,MISC
11,BANKINDIA,190.00,-30.43,52.0,H-MC,8.40,180869.0,1061.0,111343.0,1.09,0.59,61.56,62.52,88.0,0.01,1.33,34.88,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-45.59,29.0,M-SC,1.39,91988.0,490.0,78263.0,-0.49,0.53,85.08,86.07,223.0,0.01,0.68,33.58,XR,NTT,DURABLES
86,WIPRO,420.00,-16.52,35.0,M-LC,5.48,147083.0,-3862.0,113313.0,0.63,-2.56,77.04,72.51,53.0,-0.03,1.08,3.31,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,7.00,206166.0,-45602.0,98815.0,-2.55,-18.11,47.93,21.13,27.0,-0.46,1.51,10.90,X40,ATH,PAINTS
44,JIOFIN,387.00,-5.85,30.0,H-LC,12.80,255764.0,-11601.0,77062.0,0.52,-4.34,30.13,24.48,48.0,-0.15,1.88,48.25,XY24,BTT,FINANCE
74,TCS,4476.75,-31.26,30.0,H-LC,12.16,273512.0,-72444.0,147314.0,0.37,-20.94,53.86,21.64,1.0,-0.49,2.01,0.37,X40,ATH,IT
31,HINDUNILVR,2922.00,-12.79,31.0,H-LC,8.91,244900.0,-12163.0,47290.0,-2.50,-4.73,19.31,13.67,24.0,-0.26,1.80,14.71,XY25,NTT,FMCG
83,VBL,671.64,-20.64,32.0,H-LC,8.51,285696.0,-30146.0,142819.0,0.69,-9.54,49.99,35.67,5.0,-0.21,2.10,2.99,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.80,40.0,H-LC,1.57,228712.0,-17439.0,96654.0,0.15,-7.08,42.26,32.18,16.0,-0.18,1.68,26.41,X200,ATH,IT
42,ITC,452.0,-38.77,44.0,H-LC,1.93,197556.0,-2582.0,23470.0,-0.27,-1.29,11.88,10.44,4.0,-0.11,1.45,4.20,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.61,44.0,H-LC,2.39,156430.0,-29665.0,77151.0,1.04,-15.94,49.32,25.51,15.0,-0.38,1.15,15.28,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.66,47.0,H-LC,3.94,215842.0,5476.0,23311.0,0.44,2.60,10.80,13.68,37.0,0.23,1.58,19.56,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.52,39.0,H-LC,4.29,206363.0,1863.0,39560.0,0.46,0.91,19.17,20.25,10.0,0.05,1.51,9.72,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,30.0,H-LC,12.16,273512.0,-72444.0,147314.0,0.37,-20.94,53.86,21.64,1.0,-0.49,2.01,0.37,X40,ATH,IT
80,UNITDSPR,1644.00,-14.37,44.0,H-LC,7.20,224374.0,-9770.0,58405.0,0.55,-4.17,26.03,20.77,86.0,-0.17,1.65,2.46,X40N,NTT,BREWERIES
83,VBL,671.64,-20.64,32.0,H-LC,8.51,285696.0,-30146.0,142819.0,0.69,-9.54,49.99,35.67,5.0,-0.21,2.10,2.99,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.02,218199.0,-23725.0,86996.0,-0.39,-9.81,39.87,26.15,8.0,-0.27,1.60,3.11,X40,ATH,IT
42,ITC,452.00,-38.77,44.0,H-LC,1.93,197556.0,-2582.0,23470.0,-0.27,-1.29,11.88,10.44,4.0,-0.11,1.45,4.20,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,44.0,H-LC,2.39,156430.0,-29665.0,77151.0,1.04,-15.94,49.32,25.51,15.0,-0.38,1.15,15.28,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,44.0,H-LC,1.93,197556.0,-2582.0,23470.0,-0.27,-1.29,11.88,10.44,4.0,-0.11,1.45,4.20,X40,NTT,FMCG
19,CIPLA,1795.00,-22.52,39.0,H-LC,4.29,206363.0,1863.0,39560.0,0.46,0.91,19.17,20.25,10.0,0.05,1.51,9.72,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,7.00,206166.0,-45602.0,98815.0,-2.55,-18.11,47.93,21.13,27.0,-0.46,1.51,10.90,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,47.0,H-LC,3.94,215842.0,5476.0,23311.0,0.44,2.60,10.80,13.68,37.0,0.23,1.58,19.56,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.26,30.0,H-LC,12.16,273512.0,-72444.0,147314.0,0.37,-20.94,53.86,21.64,1.0,-0.49,2.01,0.37,X40,ATH,IT
40,INFY,2275.00,-20.32,42.0,H-LC,8.18,310724.0,-2164.0,173850.0,0.68,-0.69,55.95,54.87,3.0,-0.01,2.28,5.91,X40,BTT,IT
42,ITC,452.00,-38.77,44.0,H-LC,1.93,197556.0,-2582.0,23470.0,-0.27,-1.29,11.88,10.44,4.0,-0.11,1.45,4.20,X40,NTT,FMCG
83,VBL,671.64,-20.64,32.0,H-LC,8.51,285696.0,-30146.0,142819.0,0.69,-9.54,49.99,35.67,5.0,-0.21,2.10,2.99,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,47.0,H-LC,7.67,250258.0,-11361.0,130585.0,0.78,-4.34,52.18,45.57,7.0,-0.09,1.84,6.49,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-49.70,38.0,L-SC,40.02,75610.0,-37939.0,77992.0,-2.60,-33.41,103.15,35.27,268.0,-0.49,0.55,85.06,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-177.27,69.0,M-SC,6.56,88446.0,-13764.0,107152.0,6.96,-13.47,121.15,91.37,208.0,-0.13,0.65,43.58,XY25,NTT,FINANCE
49,MASFIN,398.61,-20.96,35.0,H-SC,12.29,90015.0,-7965.0,29570.0,-0.86,-8.13,32.85,22.05,152.0,-0.27,0.66,30.84,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107786.0,7282.0,21827.0,1.39,7.25,20.25,28.96,79.0,0.33,0.79,43.67,MH,ATH,METALS
78,UJJIVANSFB,60.00,87.86,44.0,H-SC,10.19,118800.0,-23679.0,43196.0,-0.41,-16.62,36.36,13.70,163.0,-0.55,0.87,40.22,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1185.49,65.0,M-SC,13.99,174740.0,-6403.0,51863.0,2.03,-3.53,29.68,25.10,235.0,-0.12,1.28,40.04,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-177.27,69.0,M-SC,6.56,88446.0,-13764.0,107152.0,6.96,-13.47,121.15,91.37,208.0,-0.13,0.65,43.58,XY25,NTT,FINANCE
81,VAIBHAVGBL,521.0,51.52,48.0,H-SC,5.75,131282.0,-51493.0,162567.0,0.12,-28.17,123.83,60.77,125.0,-0.32,0.96,17.58,XR,NTT,JEWELLERY
7,ATULAUTO,844.0,3548.65,48.0,M-SC,7.43,113658.0,-33710.0,85528.0,0.77,-22.87,75.25,35.16,236.0,-0.39,0.83,16.09,XY24,NTT,AUTO
60,SAIL,228.0,44.35,54.0,M-MC,10.71,227769.0,2807.0,164836.0,0.95,1.25,72.37,74.53,192.0,0.02,1.67,33.90,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.31
1,25,43.77
2,50,74.98


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.77
LC    32.56
MC    23.65
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.37
X40      14.78
X40N     11.43
XR       11.08
XY25     10.17
AR        8.33
OX40N     7.86
X200      1.68
MH        1.61
X5K       1.46
SR        1.21
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.28
H-LC    26.02
H-MC    20.70
M-SC    13.06
M-LC     5.48
M-MC     2.65
L-SC     1.43
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.06,-7.21,42.60
IT,12.82,-20.34,85.96
FINANCE,10.11,-19.15,71.13
MISC,6.99,-20.66,87.74
BANKS,6.16,-18.91,81.45
PAINTS,5.62,-20.46,38.42
ELECTRICAL,5.52,-9.71,48.21
INSURANCE,3.77,-7.65,47.89
AC,3.55,0.19,16.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3241956.0,22
XR,1399212.0,14
AR,1197956.0,9
X40,834444.0,10
X40N,648682.0,9
OX40N,574711.0,10
XY25,454183.0,7
SR,256521.0,2
X5K,118833.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3801573.0,29
M-SC,1466000.0,17
H-MC,1285300.0,15
H-LC,1276018.0,15
M-LC,404399.0,4
M-MC,316642.0,2
L-SC,265219.0,3
L-MC,61842.0,1
L-LC,40056.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1256137.0      6
M-SC       XY24       839427.0      7
H-SC       AR         819518.0      5
           XR         809426.0      7
H-MC       XY24       580435.0      4
H-LC       X40        530445.0      5
H-SC       X40N       319171.0      4
M-MC       XY24       316642.0      2
H-SC       OX40N      268730.0      4
           SR         256521.0      2
H-LC       X40N       240784.0      3
H-MC       X40        226504.0      4
H-LC       AR         207736.0      2
H-MC       XY25       183638.0      2
L-SC       XR         176912.0      2
M-LC       XY24       172253.0      2
M-SC       AR         170702.0      2
           OX40N      144848.0      4
           XR         126376.0      2
H-LC       XY25       123337.0      3
M-LC       X5K        118833.0      1
           XR         113313.0      1
H-MC       XR         111343.0      1
M-SC       XY25       107152.0      1
H-LC       X200        96654.0      1
H-MC       X40N        88727.0      2
L-SC       OX40N       88307.0      1
M-SC       X40         77495.0      1
H-LC       XY24        77062.0      1
H-MC       OX40N       72826.0      1
H-SC       MH          72070.0      1
L-MC       XR          61842.0      1
L-LC       XY25        40056.0      1
H-MC       MH          21827.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
